# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
data

[{'state': 'Florida',
  'shortname': 'FL',
  'info': {'governor': 'Rick Scott'},
  'counties': [{'name': 'Dade', 'population': 12345},
   {'name': 'Broward', 'population': 40000},
   {'name': 'Palm Beach', 'population': 60000}]},
 {'state': 'Ohio',
  'shortname': 'OH',
  'info': {'governor': 'John Kasich'},
  'counties': [{'name': 'Summit', 'population': 1234},
   {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
#load json as string
#json.load((open('data/world_bank_projects_less.json')))

In [8]:
#load as Pandas dataframe
#sample_json_df = pd.read_json('data/world_bank_projects_less.json')
#sample_json_df.head()

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [9]:
#json.load((open('data/world_bank_projects.json')))

In [10]:
# Use first element to see the data features
data = json.load((open('data/world_bank_projects.json')))
#data[0]

In [11]:
#obtain the information that needed for filtering
json_df = json_normalize(data, 'mjtheme_namecode', ['countryshortname', 'project_name', 'mjthemecode'])
json_df.head()

,code,name,countryshortname,project_name,mjthemecode
0,8,Human development,Ethiopia,Ethiopia General Education Quality Improvement...,"8,11"
1,11,,Ethiopia,Ethiopia General Education Quality Improvement...,"8,11"
2,1,Economic management,Tunisia,TN: DTF Social Protection Reforms Support,"1,6"
3,6,Social protection and risk management,Tunisia,TN: DTF Social Protection Reforms Support,"1,6"
4,5,Trade and integration,Tuvalu,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6"


In [12]:
json_df['code'].unique()

array(['8', '11', '1', '6', '5', '2', '7', '4', '10', '9', '3'],
      dtype=object)

In [13]:
# 10 countries with most projects
mostprojects = json_df.groupby('countryshortname').count().sort_values(by='project_name', ascending = False)
mostprojects.head(10)

,code,name,project_name,mjthemecode
countryshortname,,,,
Indonesia,56,56,56,56
India,51,51,51,51
Vietnam,43,43,43,43
Bangladesh,41,41,41,41
Brazil,41,41,41,41
China,40,40,40,40
Africa,39,39,39,39
"Yemen, Republic of",34,34,34,34
Morocco,32,32,32,32


In [14]:
dic = {}

for index, value in enumerate(json_df['name']):
    if value:
        dic[json_df['code'][index]] = value
print(dic)

for index, value in enumerate(json_df['name']):
    if not value:
        json_df['name'][index] = dic[json_df['code'][index]]

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [15]:
json_df.head(10)

,code,name,countryshortname,project_name,mjthemecode
0,8,Human development,Ethiopia,Ethiopia General Education Quality Improvement...,"8,11"
1,11,Environment and natural resources management,Ethiopia,Ethiopia General Education Quality Improvement...,"8,11"
2,1,Economic management,Tunisia,TN: DTF Social Protection Reforms Support,"1,6"
3,6,Social protection and risk management,Tunisia,TN: DTF Social Protection Reforms Support,"1,6"
4,5,Trade and integration,Tuvalu,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6"
5,2,Public sector governance,Tuvalu,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6"
6,11,Environment and natural resources management,Tuvalu,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6"
7,6,Social protection and risk management,Tuvalu,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6"
8,7,Social dev/gender/inclusion,"Yemen, Republic of",Gov't and Civil Society Organization Partnership,"7,7"
9,7,Social dev/gender/inclusion,"Yemen, Republic of",Gov't and Civil Society Organization Partnership,"7,7"


In [16]:
count_th = json_df.groupby('name').count().sort_values(by='code', ascending=False)
count_th.head(10)

,code,countryshortname,project_name,mjthemecode
name,,,,
Environment and natural resources management,250,250,250,250
Rural development,216,216,216,216
Human development,210,210,210,210
Public sector governance,199,199,199,199
Social protection and risk management,168,168,168,168
Financial and private sector development,146,146,146,146
Social dev/gender/inclusion,130,130,130,130
Trade and integration,77,77,77,77
Urban development,50,50,50,50
